## GIRR Intrabucket Correlation Matrix

In [3]:
import numpy as np
import pandas as pd
import math

In [4]:
# Regulatory Inputs

IR_tenors = np.array([0.25, 0.5, 1, 2, 3, 5, 10, 15, 20, 30])
T = len(IR_tenors)

sqrt_rule = False   # change it to True if the square root rule applies

IR_RW = np.array([0.017, 0.017, 0.016, 0.013, 0.012, 0.01, 0.01, 0.01, 0.01, 0.01])

if sqrt_rule:
    IR_RW /= np.sqrt(2)

In [5]:
# Curves & Buckets

# include curve names and currency and the mapping as needed. can be populated from a table
buckets = ['USD', 'EUR']        # e.g. 2 buckets
curves = ['C1', 'C2', 'C3']     # e.g. 3 curves ( 2 under USD bucket and 1 under EUR bucket )
IR_curve_bucket_map = [['C1','USD'], ['C2', 'USD'], ['C3', 'EUR']]

IR_df = pd.DataFrame(data = IR_curve_bucket_map , columns = ['Name', 'Bucket'])   
IR_df

,Name,Bucket
0,C1,USD
1,C2,USD
2,C3,EUR


In [10]:
# Function to generate Intrabucket Correlations

def IR_intrabucket_corr(curveBucketMap, scenario = 'Base'):
    
    # extract number of unique buckets
    buckets = curveBucketMap.Bucket.unique()
    
    # Initialize empty intrabucket correlation matrices for each bucket
    IntraBucketCorr = {}
    for i in buckets:
        
        # Fetch curve names for a given bucket
        curveNames = (curveBucketMap[curveBucketMap['Bucket'] == i])['Name']
        curve_tenor = []
        correlations = []
        for x in curveNames:
            for y in IR_tenors:
                curve_tenor.append(str(x) + "_" + str(y))
                
        
        # base correlations
        IntraBucketCorr[i] = pd.DataFrame(data = np.zeros((len(curve_tenor), len(curve_tenor))),
                                          columns = curve_tenor, index = curve_tenor)
        
        for m in range(len(curve_tenor)):
            for n in range(len(curve_tenor)):
                
                if m==n:
                     (IntraBucketCorr[i]).iloc[m,n] = 1
                elif abs(m-n)<=T-1:    # same curve , different tenor
                    (IntraBucketCorr[i]).iloc[m,n] = max(math.exp(-0.03*abs(IR_tenors[m % T]-IR_tenors[n % T])
                                                                /min(IR_tenors[m % T],IR_tenors[n % T])), 0.4)
                    
                elif abs(m-n) % T == 0:
                    (IntraBucketCorr[i]).iloc[m,n] = 0.999
                    
                else:
                    
                    (IntraBucketCorr[i]).iloc[m,n] = 0.999* max(math.exp(-0.03*abs(IR_tenors[m % T]-IR_tenors[n % T])
                                                                /min(IR_tenors[m % T],IR_tenors[n % T])), 0.4)
                    
                if scenario == "High":
                    (IntraBucketCorr[i]).iloc[m,n] = min(1, 1.25*((IntraBucketCorr[i]).iloc[m,n]))
                elif scenario == "Low":
                    (IntraBucketCorr[i]).iloc[m,n] = max(2*((IntraBucketCorr[i]).iloc[m,n])-1, 
                                                         0.75*((IntraBucketCorr[i]).iloc[m,n]))    

    return IntraBucketCorr

In [11]:
IR_intrabucket_corr(IR_df, "Base")['USD']

,C1_0.25,C1_0.5,C1_1.0,C1_2.0,C1_3.0,C1_5.0,C1_10.0,C1_15.0,C1_20.0,C1_30.0,C2_0.25,C2_0.5,C2_1.0,C2_2.0,C2_3.0,C2_5.0,C2_10.0,C2_15.0,C2_20.0,C2_30.0
C1_0.25,1.000000,0.970446,0.913931,0.810584,0.718924,0.565525,0.400000,0.400000,0.400000,0.400000,0.999000,0.969475,0.913017,0.809774,0.718205,0.564960,0.399600,0.399600,0.399600,0.399600
C1_0.5,0.970446,1.000000,0.970446,0.913931,0.860708,0.763379,0.565525,0.418952,0.400000,0.400000,0.970446,0.999000,0.969475,0.913017,0.859847,0.762616,0.564960,0.418533,0.399600,0.399600
C1_1.0,0.913931,0.970446,1.000000,0.970446,0.941765,0.886920,0.763379,0.657047,0.565525,0.418952,0.913931,0.970446,0.999000,0.969475,0.940823,0.886034,0.762616,0.656390,0.564960,0.418533
C1_2.0,0.810584,0.913931,0.970446,1.000000,0.985112,0.955997,0.886920,0.822835,0.763379,0.657047,0.810584,0.913931,0.970446,0.999000,0.984127,0.955041,0.886034,0.822012,0.762616,0.656390
C1_3.0,0.718924,0.860708,0.941765,0.985112,1.000000,0.980199,0.932394,0.886920,0.843665,0.763379,0.718924,0.860708,0.941765,0.985112,0.999000,0.979218,0.931461,0.886034,0.842821,0.762616
C1_5.0,0.565525,0.763379,0.886920,0.955997,0.980199,1.000000,0.970446,0.941765,0.913931,0.860708,0.565525,0.763379,0.886920,0.955997,0.980199,0.999000,0.969475,0.940823,0.913017,0.859847
C1_10.0,0.400000,0.565525,0.763379,0.886920,0.932394,0.970446,1.000000,0.985112,0.970446,0.941765,0.400000,0.565525,0.763379,0.886920,0.932394,0.970446,0.999000,0.984127,0.969475,0.940823
C1_15.0,0.400000,0.418952,0.657047,0.822835,0.886920,0.941765,0.985112,1.000000,0.990050,0.970446,0.400000,0.418952,0.657047,0.822835,0.886920,0.941765,0.985112,0.999000,0.989060,0.969475
C1_20.0,0.400000,0.400000,0.565525,0.763379,0.843665,0.913931,0.970446,0.990050,1.000000,0.985112,0.400000,0.400000,0.565525,0.763379,0.843665,0.913931,0.970446,0.990050,0.999000,0.984127
C1_30.0,0.400000,0.400000,0.418952,0.657047,0.763379,0.860708,0.941765,0.970446,0.985112,1.000000,0.400000,0.400000,0.418952,0.657047,0.763379,0.860708,0.941765,0.970446,0.985112,0.999000


In [12]:
IR_intrabucket_corr(IR_df, "High")['USD']

,C1_0.25,C1_0.5,C1_1.0,C1_2.0,C1_3.0,C1_5.0,C1_10.0,C1_15.0,C1_20.0,C1_30.0,C2_0.25,C2_0.5,C2_1.0,C2_2.0,C2_3.0,C2_5.0,C2_10.0,C2_15.0,C2_20.0,C2_30.0
C1_0.25,1.000000,1.000000,1.000000,1.000000,0.898655,0.706907,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,0.897756,0.706200,0.499500,0.499500,0.499500,0.499500
C1_0.5,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.706907,0.523689,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.953270,0.706200,0.523166,0.499500,0.499500
C1_1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.821309,0.706907,0.523689,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.953270,0.820487,0.706200,0.523166
C1_2.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.821309,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.953270,0.820487
C1_3.0,0.898655,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.898655,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.953270
C1_5.0,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C1_10.0,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C1_15.0,0.500000,0.523689,0.821309,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,0.523689,0.821309,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C1_20.0,0.500000,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C1_30.0,0.500000,0.500000,0.523689,0.821309,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,0.523689,0.821309,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
IR_intrabucket_corr(IR_df, "Low")['USD']

,C1_0.25,C1_0.5,C1_1.0,C1_2.0,C1_3.0,C1_5.0,C1_10.0,C1_15.0,C1_20.0,C1_30.0,C2_0.25,C2_0.5,C2_1.0,C2_2.0,C2_3.0,C2_5.0,C2_10.0,C2_15.0,C2_20.0,C2_30.0
C1_0.25,1.000000,0.940891,0.827862,0.621168,0.539193,0.424144,0.300000,0.300000,0.300000,0.300000,0.998000,0.938950,0.826035,0.619547,0.538654,0.423720,0.299700,0.299700,0.299700,0.299700
C1_0.5,0.940891,1.000000,0.940891,0.827862,0.721416,0.572535,0.424144,0.314214,0.300000,0.300000,0.940891,0.998000,0.938950,0.826035,0.719695,0.571962,0.423720,0.313899,0.299700,0.299700
C1_1.0,0.827862,0.940891,1.000000,0.940891,0.883529,0.773841,0.572535,0.492785,0.424144,0.314214,0.827862,0.940891,0.998000,0.938950,0.881646,0.772067,0.571962,0.492292,0.423720,0.313899
C1_2.0,0.621168,0.827862,0.940891,1.000000,0.970224,0.911995,0.773841,0.645669,0.572535,0.492785,0.621168,0.827862,0.940891,0.998000,0.968254,0.910083,0.772067,0.644024,0.571962,0.492292
C1_3.0,0.539193,0.721416,0.883529,0.970224,1.000000,0.960397,0.864788,0.773841,0.687330,0.572535,0.539193,0.721416,0.883529,0.970224,0.998000,0.958437,0.862923,0.772067,0.685642,0.571962
C1_5.0,0.424144,0.572535,0.773841,0.911995,0.960397,1.000000,0.940891,0.883529,0.827862,0.721416,0.424144,0.572535,0.773841,0.911995,0.960397,0.998000,0.938950,0.881646,0.826035,0.719695
C1_10.0,0.300000,0.424144,0.572535,0.773841,0.864788,0.940891,1.000000,0.970224,0.940891,0.883529,0.300000,0.424144,0.572535,0.773841,0.864788,0.940891,0.998000,0.968254,0.938950,0.881646
C1_15.0,0.300000,0.314214,0.492785,0.645669,0.773841,0.883529,0.970224,1.000000,0.980100,0.940891,0.300000,0.314214,0.492785,0.645669,0.773841,0.883529,0.970224,0.998000,0.978120,0.938950
C1_20.0,0.300000,0.300000,0.424144,0.572535,0.687330,0.827862,0.940891,0.980100,1.000000,0.970224,0.300000,0.300000,0.424144,0.572535,0.687330,0.827862,0.940891,0.980100,0.998000,0.968254
C1_30.0,0.300000,0.300000,0.314214,0.492785,0.572535,0.721416,0.883529,0.940891,0.970224,1.000000,0.300000,0.300000,0.314214,0.492785,0.572535,0.721416,0.883529,0.940891,0.970224,0.998000


In [14]:
IR_intrabucket_corr(IR_df, "Base")['EUR']

,C3_0.25,C3_0.5,C3_1.0,C3_2.0,C3_3.0,C3_5.0,C3_10.0,C3_15.0,C3_20.0,C3_30.0
C3_0.25,1.000000,0.970446,0.913931,0.810584,0.718924,0.565525,0.400000,0.400000,0.400000,0.400000
C3_0.5,0.970446,1.000000,0.970446,0.913931,0.860708,0.763379,0.565525,0.418952,0.400000,0.400000
C3_1.0,0.913931,0.970446,1.000000,0.970446,0.941765,0.886920,0.763379,0.657047,0.565525,0.418952
C3_2.0,0.810584,0.913931,0.970446,1.000000,0.985112,0.955997,0.886920,0.822835,0.763379,0.657047
C3_3.0,0.718924,0.860708,0.941765,0.985112,1.000000,0.980199,0.932394,0.886920,0.843665,0.763379
C3_5.0,0.565525,0.763379,0.886920,0.955997,0.980199,1.000000,0.970446,0.941765,0.913931,0.860708
C3_10.0,0.400000,0.565525,0.763379,0.886920,0.932394,0.970446,1.000000,0.985112,0.970446,0.941765
C3_15.0,0.400000,0.418952,0.657047,0.822835,0.886920,0.941765,0.985112,1.000000,0.990050,0.970446
C3_20.0,0.400000,0.400000,0.565525,0.763379,0.843665,0.913931,0.970446,0.990050,1.000000,0.985112
C3_30.0,0.400000,0.400000,0.418952,0.657047,0.763379,0.860708,0.941765,0.970446,0.985112,1.000000


In [15]:
IR_intrabucket_corr(IR_df, "High")['EUR']

,C3_0.25,C3_0.5,C3_1.0,C3_2.0,C3_3.0,C3_5.0,C3_10.0,C3_15.0,C3_20.0,C3_30.0
C3_0.25,1.000000,1.000000,1.000000,1.000000,0.898655,0.706907,0.500000,0.500000,0.500000,0.500000
C3_0.5,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.706907,0.523689,0.500000,0.500000
C3_1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.821309,0.706907,0.523689
C3_2.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224,0.821309
C3_3.0,0.898655,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.954224
C3_5.0,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C3_10.0,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C3_15.0,0.500000,0.523689,0.821309,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C3_20.0,0.500000,0.500000,0.706907,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
C3_30.0,0.500000,0.500000,0.523689,0.821309,0.954224,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
IR_intrabucket_corr(IR_df, "Low")['EUR']

,C3_0.25,C3_0.5,C3_1.0,C3_2.0,C3_3.0,C3_5.0,C3_10.0,C3_15.0,C3_20.0,C3_30.0
C3_0.25,1.000000,0.940891,0.827862,0.621168,0.539193,0.424144,0.300000,0.300000,0.300000,0.300000
C3_0.5,0.940891,1.000000,0.940891,0.827862,0.721416,0.572535,0.424144,0.314214,0.300000,0.300000
C3_1.0,0.827862,0.940891,1.000000,0.940891,0.883529,0.773841,0.572535,0.492785,0.424144,0.314214
C3_2.0,0.621168,0.827862,0.940891,1.000000,0.970224,0.911995,0.773841,0.645669,0.572535,0.492785
C3_3.0,0.539193,0.721416,0.883529,0.970224,1.000000,0.960397,0.864788,0.773841,0.687330,0.572535
C3_5.0,0.424144,0.572535,0.773841,0.911995,0.960397,1.000000,0.940891,0.883529,0.827862,0.721416
C3_10.0,0.300000,0.424144,0.572535,0.773841,0.864788,0.940891,1.000000,0.970224,0.940891,0.883529
C3_15.0,0.300000,0.314214,0.492785,0.645669,0.773841,0.883529,0.970224,1.000000,0.980100,0.940891
C3_20.0,0.300000,0.300000,0.424144,0.572535,0.687330,0.827862,0.940891,0.980100,1.000000,0.970224
C3_30.0,0.300000,0.300000,0.314214,0.492785,0.572535,0.721416,0.883529,0.940891,0.970224,1.000000
